In [1]:
import pandas as pd
import os

In [6]:
os.listdir()

['tmax.csv',
 'climate_dev2.csv',
 '.DS_Store',
 'climate_dev1.csv',
 'prdf.csv',
 'Reconstruct Data.ipynb',
 'Untitled.ipynb',
 'tempdata',
 'climate_changemeta2.csv',
 'finaltemplatertomerge',
 'tmin.csv',
 'climate_changemeta.csv',
 'worldbankfinal.csv',
 'tmean.csv',
 'final',
 '.ipynb_checkpoints',
 'original_from_wb',
 'climate_change.csv',
 'climate_dev_meta.csv']

First merging all of them together in case we need all of the historical data.

In [2]:
tmean = pd.read_csv(os.getcwd()+'/tmean.csv').drop(['Unnamed: 0'],axis=1)
tmax = pd.read_csv(os.getcwd()+'/tmax.csv').drop(['Unnamed: 0'],axis=1)
prdf = pd.read_csv(os.getcwd()+'/prdf.csv').drop(['Unnamed: 0'],axis=1)
tmin = pd.read_csv(os.getcwd()+'/tmin.csv').drop(['Unnamed: 0'],axis=1)

In [3]:
tmean = pd.melt(tmean, id_vars=['Date'], value_vars=tmean.columns).rename({'variable':'Country Name','value':'tmean','Date':'Year'},axis=1)
tmax = pd.melt(tmax, id_vars=['Date'], value_vars=tmax.columns).rename({'variable':'Country Name','value':'tmax','Date':'Year'},axis=1)
prdf = pd.melt(prdf, id_vars=['Date'], value_vars=prdf.columns).rename({'variable':'Country Name','value':'pr','Date':'Year'},axis=1)
tmin = pd.melt(tmin, id_vars=['Date'], value_vars=tmin.columns).rename({'variable':'Country Name','value':'tmin','Date':'Year'},axis=1)

In [6]:
tmean['CommonCol'] = tmean['Country Name'] + tmean['Year'].astype(str)
tmax['CommonCol'] = tmax['Country Name'] + tmax['Year'].astype(str)
prdf['CommonCol'] = prdf['Country Name'] + prdf['Year'].astype(str)
tmin['CommonCol'] = tmin['Country Name'] + tmin['Year'].astype(str)

In [23]:
total = tmean.merge(tmax,how='inner',on='CommonCol').drop(['Year_y','Country Name_y'],axis=1)
total = total.merge(tmin,how='inner',on='CommonCol').drop(['Year_x','Country Name_x'],axis=1)
total = total.merge(prdf,how='inner',on='CommonCol').drop(['Year_y','Country Name_y'],axis=1)

In [24]:
total.head()

,tmean,CommonCol,tmax,Year_x,Country Name_x,tmin,pr
0,15.00,Argentina1901,21.65,1901,Argentina,8.39,439.65
1,14.63,Argentina1902,21.29,1902,Argentina,8.01,495.48
2,14.30,Argentina1903,20.97,1903,Argentina,7.68,540.98
3,14.49,Argentina1904,21.15,1904,Argentina,7.87,557.03
4,14.12,Argentina1905,20.78,1905,Argentina,7.51,538.47


In [25]:
total = total[['CommonCol','Year_x','Country Name_x','tmean','tmin','tmax','pr']].rename({'Year_x':'Year','Country Name_x':'Country Name'},axis=1)
total.to_csv('temperaturetotal.csv')

Then merging onnly 1980+ temperature data which I will then merge with the world bank data.

In [26]:
tmean = pd.read_csv(os.getcwd()+'/tmean.csv').drop(['Unnamed: 0'],axis=1)
tmax = pd.read_csv(os.getcwd()+'/tmax.csv').drop(['Unnamed: 0'],axis=1)
prdf = pd.read_csv(os.getcwd()+'/prdf.csv').drop(['Unnamed: 0'],axis=1)
tmin = pd.read_csv(os.getcwd()+'/tmin.csv').drop(['Unnamed: 0'],axis=1)

In [28]:
tmean = pd.melt(tmean, id_vars=['Date'], value_vars=tmean.columns).rename({'variable':'Country Name','value':'tmean','Date':'Year'},axis=1)
tmax = pd.melt(tmax, id_vars=['Date'], value_vars=tmax.columns).rename({'variable':'Country Name','value':'tmax','Date':'Year'},axis=1)
prdf = pd.melt(prdf, id_vars=['Date'], value_vars=prdf.columns).rename({'variable':'Country Name','value':'pr','Date':'Year'},axis=1)
tmin = pd.melt(tmin, id_vars=['Date'], value_vars=tmin.columns).rename({'variable':'Country Name','value':'tmin','Date':'Year'},axis=1)

In [31]:
tmean = tmean[tmean['Year']>=1980]
tmax = tmax[tmax['Year']>=1980]
prdf = prdf[prdf['Year']>=1980]
tmin = tmin[tmin['Year']>=1980]

In [32]:
tmean['CommonCol'] = tmean['Country Name'] + tmean['Year'].astype(str)
tmax['CommonCol'] = tmax['Country Name'] + tmax['Year'].astype(str)
prdf['CommonCol'] = prdf['Country Name'] + prdf['Year'].astype(str)
tmin['CommonCol'] = tmin['Country Name'] + tmin['Year'].astype(str)

total = tmean.merge(tmax,how='inner',on='CommonCol').drop(['Year_y','Country Name_y'],axis=1)
total = total.merge(tmin,how='inner',on='CommonCol').drop(['Year_x','Country Name_x'],axis=1)
total = total.merge(prdf,how='inner',on='CommonCol').drop(['Year_y','Country Name_y'],axis=1)

/var/folders/2g/x52yhs_s6h96x64ydv7ck14m0000gn/T/ipykernel_57899/3530587110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmean['CommonCol'] = tmean['Country Name'] + tmean['Year'].astype(str)


In [35]:
total = total[['CommonCol','Year_x','Country Name_x','tmean','tmin','tmax','pr']].rename({'Year_x':'Year','Country Name_x':'Country Name'},axis=1)

In [36]:
total.to_csv('alltemperature.csv')

Now merging the two together

In [44]:
worldbank = pd.read_csv('/Users/gamzebilsen/Documents/GitHub/Group_J_Climate_Economics/data_prep/worldbank/worldbankfinal.csv').drop(['Unnamed: 0'],axis=1)
temperature = pd.read_csv('/Users/gamzebilsen/Documents/GitHub/Group_J_Climate_Economics/data_prep/worldbank/temperaturefinal.csv').drop(['Unnamed: 0'],axis=1)

In [50]:
final = temperature.merge(worldbank,how='inner',on='CommonCol').drop(['Year_y','Country Name_y'],axis=1).rename({'Year_x':'Year','Country Name_x':'Country Name'},axis=1)

In [52]:
final.to_csv('final.csv')

Creating the final final one that merges disasters with worldbank data

In [58]:
worldbank_t = pd.read_csv('/Users/gamzebilsen/Documents/GitHub/Group_J_Climate_Economics/data_prep/worldbank_total.csv').drop(['Unnamed: 0'],axis=1)
disasters_t = pd.read_csv('/Users/gamzebilsen/Documents/GitHub/Group_J_Climate_Economics/data_prep/panel_total.csv')


In [63]:
disasters_t['CommonCol'] = disasters_t['Country']+disasters_t['Start Year'].astype(int).astype(str)

In [66]:
final_total = disasters_t.merge(worldbank_t,how='right',on='CommonCol')
final_total.to_csv('final_total.csv')